<a href="https://colab.research.google.com/github/Ananya-AJ/Deep-Learning/blob/main/Assignment4/k_data_augmentation_classification__Document.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The 20 Newsgroups dataset is a collection of approximately 20,000 newsgroup documents, partitioned (nearly) evenly across 20 different newsgroups. The dataset is widely used for text classification.Some of the newsgroup categories include politics, religion, science, and sports.

In [8]:
import os
import torch
import numpy as np
import pandas as pd
import nlpaug.augmenter.word as naw
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical

In [ ]:
!wget http://qwone.com/~jason/20Newsgroups/20news-18828.tar.gz
!tar xvzf 20news-18828.tar.gz

load the "20 Newsgroups" dataset and train a simple neural network on it using PyTorch

In [3]:
import os
import torch
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.datasets import fetch_20newsgroups

# download and load the 20 Newsgroups dataset
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

# convert text data to numerical features using a bag-of-words model
vectorizer = CountVectorizer(stop_words='english', max_features=1000)
X_train = vectorizer.fit_transform(newsgroups_train.data).toarray()
X_test = vectorizer.transform(newsgroups_test.data).toarray()
y_train = newsgroups_train.target
y_test = newsgroups_test.target

# convert data to PyTorch tensors
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

# define a simple neural network model
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = torch.nn.Linear(1000, 100)
        self.fc2 = torch.nn.Linear(100, 20)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# create an instance of the neural network model
net = Net()

# define the loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.01)

# train the neural network
for epoch in range(100):
    # zero the parameter gradients
    optimizer.zero_grad()

    # forward + backward + optimize
    outputs = net(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()

    # print statistics
    if (epoch+1) % 10 == 0:
        print(f'Epoch {epoch+1}, loss: {loss.item():.4f}')

# evaluate the neural network on the test data
with torch.no_grad():
    outputs = net(X_test)
    _, predicted = torch.max(outputs, 1)
    accuracy = (predicted == y_test).sum().item() / len(y_test)
    print(f'Test accuracy: {accuracy:.4f}')


Epoch 10, loss: 2.9993
Epoch 20, loss: 2.9964
Epoch 30, loss: 2.9935
Epoch 40, loss: 2.9907
Epoch 50, loss: 2.9880
Epoch 60, loss: 2.9854
Epoch 70, loss: 2.9828
Epoch 80, loss: 2.9803
Epoch 90, loss: 2.9779
Epoch 100, loss: 2.9755
Test accuracy: 0.0659


Data Augmentation using nlpaug. naw.SynonymAug is a data augmentation technique provided by the nlpaug library. It performs word replacement by synonyms, which are words with the same or similar meanings.

In [4]:
!pip install nlpaug

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 8.2 MB/s eta 0:00:00


<class 'sklearn.utils._bunch.Bunch'>


In [18]:
import pandas as pd
import nlpaug.augmenter.word as naw
# Define the text augmentation function
aug = naw.SynonymAug(aug_src='wordnet')

augmented_data = []
for text in newsgroups_train["data"]:
    augmented_text = aug.augment(text)
    augmented_text = ' '.join(augmented_text)  # Convert list of words to string
    augmented_data.append(augmented_text)

In [19]:
# Create bag-of-words representations of the augmented text data
vectorizer = CountVectorizer(stop_words='english', max_features=10000)
train_data_bow = vectorizer.fit_transform(augmented_data)
test_data_bow = vectorizer.transform(newsgroups_test.data)

In [21]:
train_data_bow = train_data_bow.sorted_indices()

In [22]:
# Define the neural network model
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout

model = Sequential()
model.add(Dense(64, input_shape=(train_data_bow.shape[1],), activation='relu'))
model.add(Dense(20, activation='softmax'))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
batch_size = 32
epochs = 10
model.fit(train_data_bow, newsgroups_train.target, batch_size=batch_size, epochs=epochs, verbose=1)

Epoch 1/10
354/354 [==============================] - 5s 12ms/step - loss: 1.9308 - accuracy: 0.5846
Epoch 2/10
354/354 [==============================] - 5s 15ms/step - loss: 0.8984 - accuracy: 0.8253
Epoch 3/10
354/354 [==============================] - 6s 16ms/step - loss: 0.5648 - accuracy: 0.8997
Epoch 4/10
354/354 [==============================] - 5s 14ms/step - loss: 0.3794 - accuracy: 0.9318
Epoch 5/10
354/354 [==============================] - 5s 14ms/step - loss: 0.2773 - accuracy: 0.9495
Epoch 6/10
354/354 [==============================] - 5s 13ms/step - loss: 0.2184 - accuracy: 0.9598
Epoch 7/10
354/354 [==============================] - 7s 18ms/step - loss: 0.1799 - accuracy: 0.9631
Epoch 8/10
354/354 [==============================] - 4s 11ms/step - loss: 0.1584 - accuracy: 0.9661
Epoch 9/10
354/354 [==============================] - 5s 15ms/step - loss: 0.1402 - accuracy: 0.9677
Epoch 10/10
354/354 [==============================] - 6s 16ms/step - loss: 0.1292 - accura

In [23]:
test_loss, test_acc = model.evaluate(test_data_bow, newsgroups_test.target)
print('Test accuracy:', test_acc)

236/236 [==============================] - 1s 3ms/step - loss: 1.9001 - accuracy: 0.6414
Test accuracy: 0.641396701335907


As we can see test accuracy remained same but the loss reduced considerably